In [41]:
import pandas as pd
import torch
from nn import MotifCaller
from training_data import data_preproc, load_training_data
from utils import get_savepaths
import numpy as np

In [32]:
# My guy that's not how load a model
n_classes = 17
model_path = r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\synthetic\model_27_2_25.pth"

In [34]:
def load_model(model_path, device):
    """
    Loading model purely for inference
    Will need to lead optimizer to fine tune
    """
    # Model Definition
    model = MotifCaller(n_classes=17)
    
    if device == torch.device('cpu'):
        checkpoint = torch.load(model_path, map_location=torch.device('cpu'))
    else:
        checkpoint = torch.load(model_path)

    model.load_state_dict(checkpoint['model_state_dict'])

    model = model.to(device)
    return model

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
torch.set_default_device(device)
    

# Load model
# device
# port these useful methods to utils

In [35]:
model = load_model(model_path=model_path, device=device)

In [29]:
model = torch.load(r"C:\Users\Parv\Doc\HelixWorks\Basecalling\code\motifcaller\models\synthetic\model_27_2_25.pth")

In [16]:
dataset = get_savepaths(running_on_hpc=False)[0]

In [17]:
df = pd.read_pickle(dataset)
df.head()

,motif_seq,base_seq,squiggle
0,"[5, 11, 11, 7, 11, 11, 8, 11, 12, 2, 12, 12, 5...",TTTATCGTCGTCACATCAGTCGACATCAGTCGGCATGAAGACACTA...,"[504, 506, 491, 502, 504, 515, 458, 472, 458, ..."
1,"[14, 14, 8, 14, 15, 2, 15, 15, 5, 15, 15, 7, 1...",TGACGTCGGATGACGTCGGCAGCGCCACCAACTCCACAAATGACGT...,"[469, 480, 479, 479, 481, 482, 483, 498, 505, ..."
2,"[10, 10, 3, 10, 10, 7, 10, 11, 2, 11, 11, 4, 1...",GGACAGCTAGGGACAGCTACACCCCCGTATTTTGAGCGGGGGACAG...,"[465, 470, 479, 486, 539, 532, 533, 525, 527, ..."
3,"[12, 1, 12, 12, 4, 12, 12, 6, 12, 12, 7, 12, 1...",TCGCCTTCATACCCCACTAACGTAGAGTACTGCCCTTCATACCCCT...,"[485, 487, 496, 495, 488, 488, 490, 488, 524, ..."
4,"[11, 12, 3, 12, 12, 4, 12, 12, 6, 12, 12, 7, 1...",AGTCGCCTTCATACCCACCCCCGTATTTTGAGCGGCCTTCATACCC...,"[544, 556, 539, 547, 536, 534, 538, 493, 488, ..."


In [19]:
dataset_path, model_save_path, file_write_path = get_savepaths(
        running_on_hpc=False)

X, y = load_training_data(
        dataset_path, column_x='squiggle', column_y='motif_seq',
        sampling_rate=0.3)

X = data_preproc(
    X=X, window_size=1024, step_size=800, normalize_values=True)

Index(['motif_seq', 'base_seq', 'squiggle'], dtype='object')


100%|██████████| 2997/2997 [00:01<00:00, 2404.57it/s]


In [49]:
from sklearn.preprocessing import normalize
import torch.nn as nn

ctc = nn.CTCLoss(blank=0, reduction='mean', zero_infinity=True)

In [60]:
counter = 0
for x_, y_ in zip(X, y):
    x_ = torch.tensor(x_, dtype=torch.float32).to(device)
    model_output = model(x_)
    model_output = model_output.permute(0, 1, 2).view(
        model_output.shape[1] * model_output.shape[0], n_classes)
    
    label_lengths = torch.tensor(len(y_))
    y_ = torch.tensor(y_).to(device)
    
    n_timesteps = model_output.shape[0]
    input_lengths = torch.tensor(n_timesteps)
    
    loss = ctc(
        model_output, y_, input_lengths, label_lengths)
    #print(loss.item())

    if loss.item() < 2:
        print(np.exp(model_output.sum(axis=0).cpu().detach().numpy()))
    
    counter += 1
    if counter == 2000:
        break
    

In [61]:
from model_config import ModelConfig

In [62]:
model_config = ModelConfig(
    n_classes=n_classes, hidden_size=256, window_size=1024, window_step=800, train_epochs=50, device=device,
    model_save_path="", write_path="", dataset='synthetic', windows=True, sampling_rate=1.0
)

In [66]:
from training_loop import run_epoch
import torch.optim as optim
from greedy_decoder import GreedyCTCDecoder

In [67]:
optimizer = optim.Adam(model.parameters(), lr=0.001)
labels_int = np.arange(n_classes).tolist()
labels = [f"{i}" for i in labels_int] # Tokens to be fed into greedy decoder
greedy_decoder = GreedyCTCDecoder(labels=labels)

In [ ]:
result_dict = run_epoch(
    model=model, model_config=model_config, X=X, y=y, ctc=ctc,
    optimizer=optimizer, greedy_decoder=greedy_decoder)

TypeError: run_epoch() missing 2 required positional arguments: 'optimizer' and 'decoder'